In [32]:

from decimal import getcontext
from decimal import Decimal


class SourceStates: #контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict: #статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states
    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict: #статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        for char, count in states.items():
            freq_dict[char] = count / len(message)
        return freq_dict
    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict: #левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum_l = 0.0
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_l
            sum_l += freq 
        return left_borders_dict #т.е получаем для каждого символа начало его отрезка внутри [0,1]
    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)

getcontext().prec = 500 #установка точности (кол-во знаков после запятой)

class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Decimal(0.0) #левая граница отрезка кодирования c выставленной точностью
        self.right = Decimal(1.0) #правая граница отрезка кодирования c выставленной точностью
        self.source_states = SourceStates(self.message) #из объекта этого класса получаем все данные о состояниях источника (см. выше)
        
    def encode(self) -> float:
        for char in self.message:
            left_border = Decimal(self.source_states.left_borders_dict[char]) #левая граница текущего символа
            right_border = Decimal(self.source_states.left_borders_dict[char] + self.source_states.freq_states[char]) #правая граница текущего символа
            current_range = self.right - self.left #длина текущего отрезка
            self.right = self.left + current_range * right_border #усекаем отрезок в соответствии с правой границей текущего символа
            self.left = self.left + current_range * left_border #усекаем отрезок в соответствии с левой границей текущего символа
        
        return (self.left + self.right) / 2 #возвращаем середину результирующего отрезка
class ArithmeticDecode:
    def __init__(self, coded_val: float, message_len: int, source_states: SourceStates):
        self.coded_val = Decimal(coded_val) #закодированное слово с выставленной точностью
        self.message_len = message_len 
        self.source_states = source_states #из объекта этого класса получаем все данные о состояниях источника (см. выше)
        self.right = Decimal(1.0) #левая граница отрезка кодирования c выставленной точностью
        self.left = Decimal(0.0) #правая граница отрезка кодирования c выставленной точностью
        
    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        for i in range(self.message_len):
            current_range = self.right - self.left #длина текущего отрезка
            if current_range == 0: #выходим из цикла на границе точности чтобы избежать деления на ноль
                break
            scaled_value = (self.coded_val - self.left) / current_range #приводим закодированное значение к отрезку [0,1]
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char] #правая граница
                if add_left_freq <= scaled_value < add_right_freq: #попадает ли приведенное значение в отрезок текущего символа: если да, то добавляем и обновляем границы, если нет, то идём на следующую итерацию цикла for each
                    decoded_message += char
                    self.right = self.left + current_range * Decimal(add_right_freq)
                    self.left = self.left + current_range * Decimal(add_left_freq)
        return decoded_message

file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

code = ArithmeticEncode(data)
decode = ArithmeticDecode(code.encode(), len(data), SourceStates(data))
print(f"Frequencies: {code.source_states.freq_states}")
print(f"Encoded: {code.encode()}")
print(f"Decoded: {decode.decode()}")


Frequencies: {'В': 0.004761904761904762, 'с': 0.047619047619047616, 'е': 0.047619047619047616, ' ': 0.1380952380952381, 'и': 0.0380952380952381, 'м': 0.04285714285714286, 'в': 0.05238095238095238, 'о': 0.11428571428571428, 'л': 0.023809523809523808, 'ы': 0.023809523809523808, 'т': 0.05714285714285714, 'а': 0.04285714285714286, 'к': 0.023809523809523808, 'г': 0.004761904761904762, 'ф': 0.004761904761904762, 'п': 0.009523809523809525, 'р': 0.03333333333333333, 'н': 0.0380952380952381, 'у': 0.01904761904761905, '0': 0.04285714285714286, 'д': 0.04285714285714286, '2': 0.004761904761904762, '5': 0.009523809523809525, ',': 0.004761904761904762, 'ж': 0.004761904761904762, '8': 0.004761904761904762, '-': 0.004761904761904762, 'з': 0.004761904761904762, 'я': 0.02857142857142857, 'й': 0.014285714285714285, 'ч': 0.014285714285714285, '1': 0.0380952380952381, '.': 0.009523809523809525, 'Э': 0.004761904761904762}
Encoded: 0.000035658094779874371455570522398364280463288767428441380395615464679080977

In [27]:
from fractions import Fraction
from decimal import Decimal

class SourceStates:  # контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict:  # статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states

    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict:  # статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        total = len(message)
        for char, count in states.items():
            freq_dict[char] = Fraction(count, total)  #используем дроби для вероятностей
        return freq_dict

    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict:  # левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum_fraction = Fraction(0, 1)  #инициализация дроби 0/1
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_fraction
            sum_fraction += freq
        return left_borders_dict

    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)

#КОММЕНТАРИИ К АЛГОРИТМАМ КОДИРОВАНИЯ В ЛИСТИНГЕ ДЛЯ ДЕСЯТИЧНЫХ ДРОБЕЙ
class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Fraction(0, 1)  #левая граница 0/1 = 0
        self.right = Fraction(1, 1)  #правая граница 1/1 = 1
        self.source_states = SourceStates(self.message)

    def encode(self) -> Fraction:
        for char in self.message:
            left_border = self.source_states.left_borders_dict[char]
            right_border = left_border + self.source_states.freq_states[char]
            current_range = self.right - self.left
            self.right = self.left + current_range * right_border
            self.left = self.left + current_range * left_border

        return (self.left + self.right) / 2  #возвращает результат дробью

class ArithmeticDecode:
    def __init__(self, coded_val: Fraction, message_len: int, source_states: SourceStates):
        self.coded_val = coded_val
        self.message_len = message_len
        self.source_states = source_states
        self.right = Fraction(1, 1)  #левая граница 0/1 = 0
        self.left = Fraction(0, 1)  #правая граница 1/1 = 1

    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        for i in range(self.message_len):
            current_range = self.right - self.left
            if current_range == 0: 
                break
            scaled_value = (self.coded_val - self.left) / current_range
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char]
                if add_left_freq <= scaled_value < add_right_freq:
                    decoded_message += char
                    self.right = self.left + current_range * add_right_freq
                    self.left = self.left + current_range * add_left_freq
        return decoded_message

file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

code = ArithmeticEncode(data)
encoded_value = code.encode()
decode = ArithmeticDecode(encoded_value, len(data), SourceStates(data))
print(f"Frequencies: {code.source_states.freq_states}")
print(f"Encoded: {encoded_value}")
print(f"Decoded: {decode.decode()}")

Frequencies: {'В': Fraction(1, 210), 'с': Fraction(1, 21), 'е': Fraction(1, 21), ' ': Fraction(29, 210), 'и': Fraction(4, 105), 'м': Fraction(3, 70), 'в': Fraction(11, 210), 'о': Fraction(4, 35), 'л': Fraction(1, 42), 'ы': Fraction(1, 42), 'т': Fraction(2, 35), 'а': Fraction(3, 70), 'к': Fraction(1, 42), 'г': Fraction(1, 210), 'ф': Fraction(1, 210), 'п': Fraction(1, 105), 'р': Fraction(1, 30), 'н': Fraction(4, 105), 'у': Fraction(2, 105), '0': Fraction(3, 70), 'д': Fraction(3, 70), '2': Fraction(1, 210), '5': Fraction(1, 105), ',': Fraction(1, 210), 'ж': Fraction(1, 210), '8': Fraction(1, 210), '-': Fraction(1, 210), 'з': Fraction(1, 210), 'я': Fraction(1, 35), 'й': Fraction(1, 70), 'ч': Fraction(1, 70), '1': Fraction(4, 105), '.': Fraction(1, 105), 'Э': Fraction(1, 210)}
Encoded: 2716948117808364893360663494488159916143682905584904712597991649066551767951362410285215198433619987536085666957352745686245917120921130304268016736594058507856095968347050870703861220368345683575296717088264

In [30]:
file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
print(file.read())
file.close()

Все символы такого алфавита пронумерованы от 0 до 255, а каждому номеру соответствует 8-разрядный двоичный код от 00000000 до 11111111. Этот код является порядковым номером символа в двоичной системе счисления.
